# Visual Question Answering using Multimodal Transformer Models

## Import necessary libraries & set up the environment

In [1]:
from google.colab import drive

drive.mount("/content/drive")

%cd '/content/drive/MyDrive/OCR/'

!git clone https://github.com/deepdoctection/deepdoctection.git

%cd deepdoctection

!sudo apt-get install python3.8

#!pip install -r requirements.txt

!sudo apt install tesseract-ocr
!pip install pytesseract

#!pip install -q git+https://github.com/huggingface/transformers.git


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/OCR
fatal: destination path 'deepdoctection' already exists and is not an empty directory.
/content/drive/MyDrive/OCR/deepdoctection
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3.8 is already the newest version (3.8.17-1+jammy1).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.


**Install Detectron2 and deepdoc pytorch from source**

In [2]:
!pwd
!ls
!cp setup.py .
!pip install .[tf]
!pip install .[source-pt]

# !pip install tf2onnx>=1.9.2

/content/drive/MyDrive/OCR/deepdoctection
'=1.9.2'   deepdoctection	     LICENSE	  requirements.txt   tests
 build	   deepdoctection.egg-info   Makefile	  scripts	     tests_d2
 configs   docker		     mkdocs.yml   setup.cfg
 data	   docs			     README.md	  setup.py
cp: 'setup.py' and './setup.py' are the same file
Processing /content/drive/MyDrive/OCR/deepdoctection
  Preparing metadata (setup.py) ... done
  Created wheel for deepdoctection: filename=deepdoctection-0.26-py3-none-any.whl size=539674 sha256=6d7ed38aa32c207fd6c7531c0dad2f80edfea8f6d7903c43ff20876ad48e5c58
  Stored in directory: /tmp/pip-ephem-wheel-cache-me0npm5o/wheels/cb/c0/e0/328384f2ccc14d06d6b85912f0626f99d1d2cbfb49f8a4dbbc
Successfully built deepdoctection
  Attempting uninstall: deepdoctection
    Found existing installation: deepdoctection 0.26
    Uninstalling deepdoctection-0.26:
      Successfully uninstalled deepdoctection-0.26
Processing /content/drive/MyDrive/OCR/deepdoctection
  Preparing metadata (setup.py

In [31]:
!pip install tensorflow-addons

**TrOCR** **Inference**

In [3]:
from pathlib import Path
import deepdoctection as dd
import os

[0729 20:33.43 @utils.py:160]  INF  NumExpr defaulting to 2 threads.


**Catalog and Registries**

In [6]:
dd.print_model_infos(add_description=False,add_config=False,add_categories=False)

╒═════════════════════════════════════════════════════════════════════╕
│ name                                                                │
╞═════════════════════════════════════════════════════════════════════╡
│ layout/model-800000_inf_only.data-00000-of-00001                    │
├─────────────────────────────────────────────────────────────────────┤
│ cell/model-1800000_inf_only.data-00000-of-00001                     │
├─────────────────────────────────────────────────────────────────────┤
│ item/model-1620000_inf_only.data-00000-of-00001                     │
├─────────────────────────────────────────────────────────────────────┤
│ item/model-1620000.data-00000-of-00001                              │
├─────────────────────────────────────────────────────────────────────┤
│ layout/model-800000.data-00000-of-00001                             │
├─────────────────────────────────────────────────────────────────────┤
│ cell/model-1800000.data-00000-of-00001                        

**Instantiate Textline Detector**

doctr/db_resnet50/pt/db_resnet50-ac60cadc.pt - used for word detection

doctr/crnn_vgg16_bn/pt/crnn_vgg16_bn-9762b0b0.pt - used for text recognition

In [13]:
categories=dd.ModelCatalog.get_profile("doctr/db_resnet50/pt/db_resnet50-ac60cadc.pt").categories

print(dd.ModelCatalog.get_profile("doctr/db_resnet50/pt/db_resnet50-ac60cadc.pt").model_wrapper)

path_weights=dd.ModelDownloadManager.maybe_download_weights_and_configs("doctr/db_resnet50/pt/db_resnet50-ac60cadc.pt")

# doctr/crnn_vgg16_bn/pt/crnn_vgg16_bn-9762b0b0.pt

[0729 21:23.52 @fs.py:94]  INF  File db_resnet50-ac60cadc.pt exists! Skip download.


DoctrTextlineDetector


**Instantiate Text Recognizer**

**Doctr** OR **Tesseract** can be used for this purpose

Use one and comment other code

In [14]:
#Doctr
categories_tr=dd.ModelCatalog.get_profile("doctr/crnn_vgg16_bn/pt/crnn_vgg16_bn-9762b0b0.pt").categories

print(dd.ModelCatalog.get_profile("doctr/crnn_vgg16_bn/pt/crnn_vgg16_bn-9762b0b0.pt").model_wrapper)

path_weights_tr=dd.ModelDownloadManager.maybe_download_weights_and_configs("doctr/crnn_vgg16_bn/pt/crnn_vgg16_bn-9762b0b0.pt")

#Tesseract

# tess_ocr_config_path = dd.get_configs_dir_path() / "dd/conf_tesseract.yaml"  # This file will be in your .cache if you ran the analyzer before.
# # Otherwise make sure to copy the file from 'configs/conf_tesseract.yaml'

# tesseract_ocr = dd.TesseractOcrDetector(tess_ocr_config_path.as_posix())

[0729 21:23.53 @fs.py:94]  INF  File crnn_vgg16_bn-9762b0b0.pt exists! Skip download.


DoctrTextRecognizer


In [19]:
from pathlib import Path
import deepdoctection as dd
import os
# import tensorflow_addons as tfa

text_line_predictor = dd.DoctrTextlineDetector("db_resnet50",path_weights, categories)
layout = dd.ImageLayoutService(text_line_predictor,
                            to_image=True)     # ImageAnnotation created from this service will get a nested image
                                                # defined by the bounding boxes of its annotation. This is helpful
                                                # if you want to call a service only on the region of the
                                                # ImageAnnotation

text_recognizer = dd.DoctrTextRecognizer("crnn_vgg16_bn", path_weights_tr)
text = dd.TextExtractionService(text_recognizer, extract_from_roi="word") # text recognition on the region of word
                                                                        # ImageAnnotation
analyzer = dd.DoctectionPipe(pipeline_component_list=[text])      # defining the pipeline


file_path = os.path.join("data","Good", "T-40411", "T-40411a.JPG")

df = analyzer.analyze(path=file_path)
print(df)
# SerializerJsonlines.save(df, path= "path/to",
#                               file_name="doc.jsonl",
#                               max_datapoints=20)

[0729 21:42.57 @lstm.py:588]  DBG  Layer lstm will use cuDNN kernels when running on GPU.
[0729 21:42.57 @lstm.py:588]  DBG  Layer lstm will use cuDNN kernels when running on GPU.
[0729 21:42.57 @lstm.py:588]  DBG  Layer lstm will use cuDNN kernels when running on GPU.
[0729 21:42.57 @lstm.py:588]  DBG  Layer lstm_1 will use cuDNN kernels when running on GPU.
[0729 21:42.57 @lstm.py:588]  DBG  Layer lstm_1 will use cuDNN kernels when running on GPU.
[0729 21:42.57 @lstm.py:588]  DBG  Layer lstm_1 will use cuDNN kernels when running on GPU.


  0%|          | 0/58758994 [00:00<?, ?it/s]

AttributeError: ignored